In [1]:
# 0) Imports & config
import pandas as pd, numpy as np
from pathlib import Path

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

CSV_PATH = "branch_cash_data.csv"   # <-- change me
TEST_DAYS = 90                      # last N days = test window

In [2]:
# 1) Me being me -- Load + quick check *** You don't need this prod/real run
# df = pd.read_csv(CSV_PATH, parse_dates=["date"]) # works but old way
df = pd.read_csv(
    CSV_PATH,
    parse_dates=["date"],
    date_format="%Y-%m-%d",          # <-- removes the warning & speeds up parsing. new Micah way
    dtype={
        "branch_id": "string",
        "atm_withdrawals": "float64",
        "otc_withdrawals": "float64",
        "cash_deposits": "float64",
        # use nullable int so NaNs are allowed:
        "is_holiday": "Int8",
        "is_payday": "Int8",
        "is_branch_open": "Int8",
    }
)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20160 entries, 0 to 20159
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   branch_id        20160 non-null  string 
 1   date             20160 non-null  object 
 2   atm_withdrawals  19758 non-null  float64
 3   otc_withdrawals  19758 non-null  float64
 4   is_holiday       20160 non-null  Int8   
 5   is_payday        20160 non-null  Int8   
 6   cash_deposits    20160 non-null  float64
 7   is_branch_open   20060 non-null  Int8   
dtypes: Int8(3), float64(3), object(1), string(1)
memory usage: 905.8+ KB


,branch_id,date,atm_withdrawals,otc_withdrawals,is_holiday,is_payday,cash_deposits,is_branch_open
0,B004,5/10/23,18942.85,21388.36,0,0,13910.23,1
1,B002,5/16/25,30546.53,28107.67,0,0,25078.09,1
2,B008,11/15/18,19224.35,15983.56,0,1,20045.88,1
3,B002,8/22/23,34942.60,22944.67,0,0,19811.85,1
4,B007,8/7/22,13652.40,9779.20,0,0,6030.40,1


In [3]:
# Normalize date strings (replace en/em/minus dashes with hyphen) and parse strictly

# Load (or keep your existing df) and normalize the date strings
df["date"] = (
    df["date"].astype(str).str.strip()
      .str.replace(r"[–—−]", "-", regex=True)  # normalize odd dashes
)

# OPTION A: if your pandas >= 2.2 supports mixed parsing, this is simplest:
try:
    df["date"] = pd.to_datetime(df["date"], format="mixed", errors="raise")
except Exception as e:
    print("mixed parse failed, falling back to explicit formats:", e)
    # OPTION B: explicit formats by mask (no warnings, still strict)
    s = df["date"]
    iso_mask  = s.str.fullmatch(r"\d{4}-\d{2}-\d{2}")
    us4_mask  = s.str.fullmatch(r"\d{1,2}/\d{1,2}/\d{4}")
    us2_mask  = s.str.fullmatch(r"\d{1,2}/\d{1,2}/\d{2}")

    parsed = pd.Series(pd.NaT, index=df.index, dtype="datetime64[ns]")
    parsed.loc[iso_mask] = pd.to_datetime(s.loc[iso_mask], format="%Y-%m-%d")
    parsed.loc[us4_mask] = pd.to_datetime(s.loc[us4_mask], format="%m/%d/%Y")
    parsed.loc[us2_mask] = pd.to_datetime(s.loc[us2_mask], format="%m/%d/%y")

    # anything left: last-chance permissive parse, then report
    rem = parsed.isna()
    if rem.any():
        parsed.loc[rem] = pd.to_datetime(s.loc[rem], errors="coerce")
    bad = parsed.isna()
    if bad.any():
        print(f"Unparsable date rows: {bad.sum()}")
        display(df.loc[bad, ["branch_id","date"]].head(10))
        # drop or fix as you prefer:
        df = df.loc[~bad].copy()

    df["date"] = parsed

# continue: dedupe/order, dtypes
df = (
    df.drop_duplicates(subset=["branch_id","date"])
      .sort_values(["branch_id","date"])
      .reset_index(drop=True)
)

for col in ["is_holiday","is_payday","is_branch_open"]:
    if col not in df.columns: df[col] = 0
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype("Int8")

for col in ["atm_withdrawals","otc_withdrawals","cash_deposits"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("float64")

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   branch_id        20000 non-null  string        
 1   date             20000 non-null  datetime64[ns]
 2   atm_withdrawals  19600 non-null  float64       
 3   otc_withdrawals  19600 non-null  float64       
 4   is_holiday       20000 non-null  Int8          
 5   is_payday        20000 non-null  Int8          
 6   cash_deposits    20000 non-null  float64       
 7   is_branch_open   20000 non-null  Int8          
dtypes: Int8(3), datetime64[ns](1), float64(3), string(1)
memory usage: 898.6 KB


,branch_id,date,atm_withdrawals,otc_withdrawals,is_holiday,is_payday,cash_deposits,is_branch_open
0,B001,2018-10-01,30239.18,23064.03,0,1,19192.04,1
1,B001,2018-10-02,26297.23,18964.94,0,0,14941.69,1
2,B001,2018-10-03,23771.65,26629.13,0,0,17430.31,1
3,B001,2018-10-04,32967.64,21965.25,0,0,27473.43,1
4,B001,2018-10-05,27759.52,15903.84,0,0,19177.98,1


In [4]:
# bad data. drop duplicates.
# you get a unique, chronologically ordered branch_id date table 
# with a clean, sequential index
# This would be perfect for time-based features.
df = df.drop_duplicates().sort_values(["branch_id","date"]).reset_index(drop=True)
df.head()

,branch_id,date,atm_withdrawals,otc_withdrawals,is_holiday,is_payday,cash_deposits,is_branch_open
0,B001,2018-10-01,30239.18,23064.03,0,1,19192.04,1
1,B001,2018-10-02,26297.23,18964.94,0,0,14941.69,1
2,B001,2018-10-03,23771.65,26629.13,0,0,17430.31,1
3,B001,2018-10-04,32967.64,21965.25,0,0,27473.43,1
4,B001,2018-10-05,27759.52,15903.84,0,0,19177.98,1


In [5]:
# 2) Handle nulls in withdrawals, respect closures if present

# if the branch was closed that day, treat withdrawals as truly zero
# don’t impute them (in real English - don't fill them up with average/whatever). 

for col in ["atm_withdrawals","otc_withdrawals"]:
    if col not in df.columns:
        raise ValueError(f"Missing column: {col}")

# if you have an is_branch_open column, set closed-day withdrawals to 0
if "is_branch_open" in df.columns:
    closed = df["is_branch_open"]==0
    df.loc[closed, ["atm_withdrawals","otc_withdrawals"]] = 0.0

# branch-wise robust impute: 14-day rolling median, then branch median

# Uses a 14-day trailing rolling median to fill NaNs.
# Trailing window = uses past values only (NaNs are ignored), so no leakage.
# Median is robust to spikes (better than mean for cash bursts).
# min_periods=1 lets the window work at the start of the series.
        
def impute_series(s):
    s2 = s.copy()
    s2 = s2.fillna(s2.rolling(14, min_periods=1).median()) # recent history fill
    s2 = s2.fillna(s2.median()) # fallback - If there are still gaps (e.g., very early days with no history), 
                                # fill them with the branch’s overall median.
                                # Why this pattern? recent, robust estimate first (captures weekly/month-end rhythm), 
                                # then a safe branch-level fallback to avoid leaving NaNs.
    return s2

for col in ["atm_withdrawals","otc_withdrawals"]:
    df[col] = df.groupby("branch_id", group_keys=False)[col].apply(impute_series)

# Compute cash_out
# It builds a clean total cash-out number and enforces a business rule:
# clip(lower=0) on each column -> forces no negative withdrawals 
# (protects against data glitches/rollbacks that would make ATM/OTC go < 0).
# Sum the clipped ATM + OTC -> cash_out = total cash withdrawn that day.
df["cash_out"] = df["atm_withdrawals"].clip(lower=0) + df["otc_withdrawals"].clip(lower=0)

In [6]:
# 3) Calendar features (derived from date; all known at prediction time)

# — Ensure 'date' is true datetime —
df["date"] = pd.to_datetime(
    df["date"].astype(str).str.strip(),   # trim stray spaces
    format="%Y-%m-%d",                    # our generator uses ISO dates
    errors="coerce"                       # bad rows -> NaT
)
bad = df["date"].isna()
if bad.any():
    print(f"Warning: dropped {bad.sum()} rows with unparsable dates")
    df = df.loc[~bad].copy()

# — Derive features —
df["dow"] = df["date"].dt.day_name()
df["is_weekend"] = df["date"].dt.dayofweek.isin([5, 6]).astype(int)

# "last two days of month" flag (like our generator logic)
df["is_month_end"] = (
    ((df["date"] + pd.offsets.Day(1)).dt.month != df["date"].dt.month) |
    ((df["date"] + pd.offsets.Day(2)).dt.month != df["date"].dt.month)
).astype(int)

# — Ensure holiday/payday columns exist and are clean ints —
for col in ["is_holiday", "is_payday"]:
    if col not in df.columns:
        df[col] = 0
    else:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype("int8")

In [7]:
# 4) Lags & rolling means per branch (NO leakage) + target y_next
#    Vectorized version (no groupby.apply warning)

# --- safety checks ---
required = ["branch_id", "date", "cash_out"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns needed for features: {missing}")

# Keep rows ordered by branch & date
df = df.sort_values(["branch_id", "date"]).copy()

# Make 'dow' categorical (helps memory / one-hot later)
if "dow" in df.columns and df["dow"].dtype != "category":
    df["dow"] = df["dow"].astype("category")

grp = df.groupby("branch_id", sort=False, group_keys=False)

# Past-only features (shift ensures no leakage)
df["lag1"]  = grp["cash_out"].shift(1)
df["lag7"]  = grp["cash_out"].shift(7)
df["ma7"]   = grp["cash_out"].transform(lambda s: s.rolling(7,  min_periods=7).mean().shift(1))
df["ma28"]  = grp["cash_out"].transform(lambda s: s.rolling(28, min_periods=28).mean().shift(1))

# Target: tomorrow's cash_out
df["y_next"] = grp["cash_out"].shift(-1)

# Drop rows that don't have enough history or target
feat_cols = ["lag1","lag7","ma7","ma28","is_holiday","is_payday","is_month_end","is_weekend","dow"]
dfm = df.dropna(subset=feat_cols + ["y_next"]).reset_index(drop=True)

print(f"Training rows after feature build: {len(dfm):,} (from {len(df):,})")
dfm[["branch_id","date"] + feat_cols + ["y_next"]].head()

Training rows after feature build: 19,768 (from 20,000)


,branch_id,date,lag1,lag7,ma7,ma28,is_holiday,is_payday,is_month_end,is_weekend,dow,y_next
0,B001,2018-10-29,33490.37,38584.57,40147.525714,38944.082143,0,0,0,0,Monday,49674.84
1,B001,2018-10-30,42620.57,41899.62,40724.097143,38562.559286,0,0,1,0,Tuesday,44740.41
2,B001,2018-10-31,49674.84,36376.76,41834.842857,38720.154643,0,0,1,0,Wednesday,46302.39
3,B001,2018-11-01,44740.41,45553.33,43029.650000,38517.998571,0,1,0,0,Thursday,52392.41
4,B001,2018-11-02,46302.39,44022.65,43136.658571,38209.766429,0,0,0,0,Friday,35792.12


In [8]:
# Feature Engineering Rule

# Here's the simple (sort of) rule I use on any problem:
# 1) State the target & timing.
#    What are you predicting, and when is the prediction made?
#    Only use data you'd know by that time (no future info).
# 2) List what you have.
#    Columns, IDs, timestamps, categories. Circle the ones available at prediction time.
# 3) Pick 1-2 candidates from proven templates:
#    - Time/sequence: lags (y_{t-1}), rolling mean/std (past-only), day-of-week/season.
#    - Tabular: counts, rates, ratios (x1/x2, x1-x2), missing-value flags.
#    - Categoricals: one-hot (low cardinality) or count/target encoding (high).
#    - Text/images: simple length/TF-IDF or pretrained embeddings.
# 4) Engineer safely.
#    Compute with past data only; add a quick "leakage check" ("Would I know this at prediction time?").
# 5) Test, don't guess.
#    Keep a fixed validation split. Train your baseline, then add the new feature(s) and retrain.
# 6) Decide by lift.
#    If your metric improves consistently (e.g., MAPE ↓ by >= 0.3–0.5, AUC ↑ a bit), keep it; otherwise drop it.
# 7) Repeat in small steps.
#    Add another candidate, test again. Stop when gains flatten.
# 8) Favor simple & stable.
#    Prefer features that are easy to compute, explain, and won't drift.
#
# start simple -> add one feature -> measure -> keep or toss.

In [9]:
# 5) Time-based split
last_date = dfm["date"].max()
test_start = last_date - pd.Timedelta(days=TEST_DAYS)

train_mask = dfm["date"] <= test_start
test_mask  = dfm["date"] >  test_start

X_train, y_train = dfm.loc[train_mask, feat_cols], dfm.loc[train_mask, "y_next"]
X_test,  y_test  = dfm.loc[test_mask,  feat_cols], dfm.loc[test_mask,  "y_next"]

len(X_train), len(X_test), last_date, test_start

(19055,
 713,
 Timestamp('2025-08-04 00:00:00'),
 Timestamp('2025-05-06 00:00:00'))

In [10]:
# 6) Baselines — safe with zeros in the data
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error

# Baseline predictions
naive_pred = X_test["lag1"].to_numpy()   # yesterday's value
ma7_pred   = X_test["ma7"].to_numpy()    # 7-day average
y_true     = y_test.to_numpy()

# Helper metrics
def mape_safe(y, yhat, eps=1e-6):
    """MAPE that never divides by zero."""
    y = np.asarray(y)
    return np.mean(np.abs(y - yhat) / np.maximum(np.abs(y), eps))

def wmape(y, yhat, eps=1e-6):
    """Weighted MAPE (sum abs error / sum actual)."""
    return float(np.sum(np.abs(y - yhat)) / max(np.sum(np.abs(y)), eps))

def mae(y, yhat):
    """Mean Absolute Error."""
    return float(np.mean(np.abs(y - yhat)))

# Also compute MAPE only where actual > 0
nz = y_true > 0
zero_days = int((~nz).sum())

baseline_results = {
    # MAPE on non-zero actuals only
    "Naive lag1 MAPE (non-zero only)": mean_absolute_percentage_error(y_true[nz], naive_pred[nz]),
    "MA7 MAPE (non-zero only)":        mean_absolute_percentage_error(y_true[nz], ma7_pred[nz]),

    # Safe MAPE with epsilon
    "Naive lag1 MAPE (safe)": mape_safe(y_true, naive_pred),
    "MA7 MAPE (safe)":        mape_safe(y_true, ma7_pred),

    # Zero-friendly metrics
    "Naive lag1 MAE":   mae(y_true, naive_pred),
    "MA7 MAE":          mae(y_true, ma7_pred),
    "Naive lag1 WMAPE": wmape(y_true, naive_pred),
    "MA7 WMAPE":        wmape(y_true, ma7_pred),

    "Zero-actual days in test": zero_days
}
baseline_results

{'Naive lag1 MAPE (non-zero only)': 0.29749628743549883,
 'MA7 MAPE (non-zero only)': 0.1931552218044746,
 'Naive lag1 MAPE (safe)': 1176637756.2490463,
 'MA7 MAPE (safe)': 1206596553.9840295,
 'Naive lag1 MAE': 12090.137896213184,
 'MA7 MAE': 8465.736792226007,
 'Naive lag1 WMAPE': 0.3159503168270949,
 'MA7 WMAPE': 0.22123421954652597,
 'Zero-actual days in test': 22}

In [11]:
# Think of these baselines as the "dumb but strong" yardsticks you must beat.
#
# What they are:
# - lag1 = "predict tomorrow = yesterday."
# - MA7  = "predict tomorrow = average of last 7 days."
#
# Why you need them:
# 1) Set the bar. If your fancy model can't beat these simple rules, it's not ready.
# 2) Catch data issues. If baselines look crazy (or way too good), your data/splits may be wrong.
# 3) Provide a fallback. In production, you can always use MA7/lag1 if the model is down.
#
# What the numbers mean (from the cell):
# - MAE  -> average dollar error (easy to explain).
# - WMAPE -> average percent error that works even when actual=0 (e.g., 0.18 = 18%).
# - MAPE (non-zero only / safe) -> percent error, but either skips zeros or protects against divide-by-zero.
# - Zero-actual days -> how many test days had actual=0; explains why plain MAPE can explode.
#
# How it helps decision-making:
# - Compare model vs. baselines on WMAPE/MAE.
# - If your model is 3–5 percentage points better (e.g., baseline WMAPE 0.20 -> model 0.15), that's meaningful.
# - If not, improve data cleaning/features—or just use MA7/lag1 for now.
#
# Baselines tell you "what good means." Your model must clearly beat lag1/MA7 on WMAPE/MAE; otherwise, 
# keep iterating.

In [12]:
# 7) Simple model: ElasticNet (interpretable, solid baseline)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error
import numpy as np

num_cols = ["lag1","lag7","ma7","ma28","is_holiday","is_payday","is_month_end","is_weekend"]
cat_cols = ["dow"]

pre = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)
pipe = Pipeline([("pre", pre), ("model", model)])
pipe.fit(X_train, y_train)

pred = pipe.predict(X_test)
y_true = y_test.to_numpy()

# --- metrics helpers (from earlier cell) ---
def mape_safe(y, yhat, eps=1e-6):
    y = np.asarray(y)
    return float(np.mean(np.abs(y - yhat) / np.maximum(np.abs(y), eps)))

def wmape(y, yhat, eps=1e-6):
    return float(np.sum(np.abs(y - yhat)) / max(np.sum(np.abs(y)), eps))

def mae(y, yhat):
    return float(np.mean(np.abs(y - yhat)))

nz = y_true > 0  # non-zero actuals

model_results = {
    "Model MAE":                mae(y_true, pred),
    "Model WMAPE":              wmape(y_true, pred),
    "Model MAPE (non-zero)":    mean_absolute_percentage_error(y_true[nz], pred[nz]),
    "Model MAPE (safe)":        mape_safe(y_true, pred),
    "RMSE":                     float(root_mean_squared_error(y_true, pred)),  # <-- no deprecation
}

# if you computed baseline_results earlier, merge to compare:
{**model_results, **baseline_results}

{'Model MAE': 5904.315540470965,
 'Model WMAPE': 0.15429686424364403,
 'Model MAPE (non-zero)': 0.1166368279656849,
 'Model MAPE (safe)': 1229033101.0387259,
 'RMSE': 12491.552693175841,
 'Naive lag1 MAPE (non-zero only)': 0.29749628743549883,
 'MA7 MAPE (non-zero only)': 0.1931552218044746,
 'Naive lag1 MAPE (safe)': 1176637756.2490463,
 'MA7 MAPE (safe)': 1206596553.9840295,
 'Naive lag1 MAE': 12090.137896213184,
 'MA7 MAE': 8465.736792226007,
 'Naive lag1 WMAPE': 0.3159503168270949,
 'MA7 WMAPE': 0.22123421954652597,
 'Zero-actual days in test': 22}

In [13]:
# Model WMAPE ~= 0.145 -> 14.5% average percent error.
# Baselines: lag1 WMAPE ~= 31.6%, MA7 WMAPE ~= 22.1% -> model beats:
#   - lag1 by ~17 percentage points
#   - MA7 by ~7.6 points (~34% relative improvement) 
# Model MAPE (non-zero) ~= 11.7% -> percent error only on days with actual > 0.
#   (Baselines: lag1 ~= 29.7%, MA7 ~= 19.3%.)
# MAE ~= $5,904 -> average dollar mistake per branch-day.
# To judge scale, compare to the average actual:
#   y_mean = y_test.mean()
#   5904 / y_mean   # ~= WMAPE

#
# Rule of thumb (varies by use case):
# WMAPE ≥ 20%: weak
# 15–20%: okay
# 10–15%: good
# <10%: excellent (I also start to doubt here)

In [14]:
import numpy as np

def wmape(y, yhat, eps=1e-6):
    return float(np.sum(np.abs(y - yhat)) / max(np.sum(np.abs(y)), eps))

def mae(y, yhat):
    return float(np.mean(np.abs(y - yhat)))

y_true = y_test.to_numpy()
y_hat  = pred  # from your model

model_wmape = wmape(y_true, y_hat)
model_mae   = mae(y_true, y_hat)

# “Accuracy%” (common business reporting): 100 * (1 - WMAPE)
forecast_accuracy_pct = 100 * (1 - model_wmape)

print({
    "WMAPE": round(model_wmape, 4),
    "Accuracy % (1 - WMAPE)": round(forecast_accuracy_pct, 1),
    "MAE ($)": round(model_mae, 2),
})

{'WMAPE': 0.1543, 'Accuracy % (1 - WMAPE)': 84.6, 'MAE ($)': 5904.32}


In [15]:
# 8) Next-day forecast per branch with P50 and P90
# P50 = model point forecast (median). 
# P90 = P50 with a safety buffer based on the 90th percentile of recent relative errors.

import numpy as np
import pandas as pd

feat_cols = ["lag1","lag7","ma7","ma28","is_holiday","is_payday","is_month_end","is_weekend","dow"]

# --- 1) Estimate a P90 buffer from the held-out set ---
y_val   = y_test.to_numpy()
yhat_val = pipe.predict(X_test)

# use only rows with non-zero actuals to avoid divide-by-zero
mask = y_val > 0
if mask.sum() >= 30:
    rel_err = np.abs(y_val[mask] - yhat_val[mask]) / y_val[mask]
    p90_rel = float(np.quantile(rel_err, 0.90))
else:
    p90_rel = 0.25  # fallback buffer if too few rows

# clamp for sanity (5%..100%)
p90_rel = float(min(max(p90_rel, 0.05), 1.00))
print(f"Using P90 buffer (90th pct relative error) = {p90_rel:.1%}")

# --- 2) Build next-day features from the latest available day per branch ---
ready = df.dropna(subset=["lag1","lag7","ma7","ma28"]).copy()
last_per_branch = ready.sort_values("date").groupby("branch_id").tail(1)

next_X = last_per_branch[feat_cols]
p50 = pipe.predict(next_X)
p50 = np.clip(p50, 0, None)  # no negative cash

# P90 as multiplicative uplift of P50
p90 = p50 * (1 + p90_rel)

# optional: round to nearest $100 for operational use
p50 = (p50 / 100).round() * 100
p90 = (p90 / 100).round() * 100

forecast = last_per_branch[["branch_id","date"]].copy()
forecast["predict_for"] = forecast["date"] + pd.Timedelta(days=1)
forecast["y_hat_P50"] = p50
forecast["y_hat_P90"] = p90

forecast = forecast[["branch_id","predict_for","y_hat_P50","y_hat_P90"]] \
           .sort_values("branch_id").reset_index(drop=True)

print("Max date in file:", df["date"].max().date())
forecast.head(20)

Using P90 buffer (90th pct relative error) = 21.6%
Max date in file: 2025-08-05


,branch_id,predict_for,y_hat_P50,y_hat_P90
0,B001,2025-08-05,48000.0,58300.0
1,B002,2025-08-05,56700.0,69000.0
2,B003,2025-08-05,51900.0,63100.0
3,B004,2025-08-05,41200.0,50100.0
4,B005,2025-08-05,38200.0,46500.0
5,B006,2025-08-05,40600.0,49300.0
6,B007,2025-08-05,36100.0,43900.0
7,B008,2025-08-06,32500.0,39500.0


In [ ]:
# === Interactive forecast for a chosen branch & date (P50 + P90) ===
import numpy as np
import pandas as pd
from datetime import date, timedelta

# ---- P90 buffer from validation (run after model is trained and you have X_test, y_test) ----
y_val = y_test.to_numpy()
yhat_val = pipe.predict(X_test)
mask = y_val > 0
if mask.sum() >= 30:
    rel_err = np.abs(y_val[mask] - yhat_val[mask]) / y_val[mask]
    P90_BUFFER_REL = float(np.quantile(rel_err, 0.90))
else:
    P90_BUFFER_REL = 0.25  # fallback if not enough data
# clamp 5%..100% for sanity
P90_BUFFER_REL = float(min(max(P90_BUFFER_REL, 0.05), 1.00))
print(f"P90 buffer (90th pct relative error) = {P90_BUFFER_REL:.1%}")

# --- simple calendar helpers (future-known) ---
def _nth_weekday_of_month(year, month, weekday, n):
    d = pd.Timestamp(year=year, month=month, day=1)
    add = (weekday - d.weekday()) % 7
    return (d + pd.Timedelta(days=add) + pd.Timedelta(weeks=n-1)).date()

def _last_weekday_of_month(year, month, weekday):
    d = (pd.Timestamp(year=year, month=month, day=1) + pd.offsets.MonthEnd(1))
    back = (d.weekday() - weekday) % 7
    return (d - pd.Timedelta(days=back)).date()

def us_bank_holidays(year:int):
    hol = {
        date(year,1,1), date(year,6,19), date(year,7,4),
        date(year,11,11), date(year,12,25)
    }
    hol.add(_nth_weekday_of_month(year,1,0,3))   # MLK
    hol.add(_nth_weekday_of_month(year,2,0,3))   # Presidents
    hol.add(_last_weekday_of_month(year,5,0))    # Memorial
    hol.add(_nth_weekday_of_month(year,9,0,1))   # Labor
    hol.add(_nth_weekday_of_month(year,11,3,4))  # Thanksgiving
    return hol

_holiday_cache = {}

def cal_feats_for(ts: pd.Timestamp):
    d = ts.date()
    y = d.year
    if y not in _holiday_cache:
        _holiday_cache[y] = us_bank_holidays(y)
    is_holiday = int(d in _holiday_cache[y])
    is_payday  = int(d.day in (1, 15))
    is_weekend = int(ts.dayofweek in (5,6))
    is_month_end = int(
        (ts + pd.Timedelta(days=1)).month != ts.month or
        (ts + pd.Timedelta(days=2)).month != ts.month
    )
    dow = ts.day_name()
    return {"is_holiday": is_holiday, "is_payday": is_payday,
            "is_weekend": is_weekend, "is_month_end": is_month_end, "dow": dow}

def predict_for_branch_date(branch_id: str, target_date_str: str):
    # --- inputs & validation ---
    try:
        target_ts = pd.to_datetime(target_date_str, format="%Y-%m-%d")
    except Exception:
        target_ts = pd.to_datetime(target_date_str)
    if branch_id not in set(df["branch_id"].astype(str).unique()):
        raise ValueError(f"Unknown branch_id '{branch_id}'. Available: {sorted(df['branch_id'].unique())[:10]} ...")

    hist = (
        df.loc[df["branch_id"].astype(str)==branch_id, ["date","cash_out"]]
          .sort_values("date").reset_index(drop=True)
    )
    if hist.empty:
        raise ValueError(f"No history for branch {branch_id}")

    last_known = hist["date"].max()
    recent_vals = list(hist.tail(28)["cash_out"].astype(float).values)

    def make_feature_row(asof_ts: pd.Timestamp):
        lag1 = recent_vals[-1] if len(recent_vals) >= 1 else np.nan
        lag7 = recent_vals[-7] if len(recent_vals) >= 7 else lag1
        ma7  = float(np.mean(recent_vals[-7:]))  if len(recent_vals) >= 2 else lag1
        ma28 = float(np.mean(recent_vals[-28:])) if len(recent_vals) >= 2 else lag1
        c = cal_feats_for(asof_ts + pd.Timedelta(days=1))
        return pd.DataFrame([{
            "lag1": lag1, "lag7": lag7, "ma7": ma7, "ma28": ma28,
            "is_holiday": c["is_holiday"], "is_payday": c["is_payday"],
            "is_month_end": c["is_month_end"], "is_weekend": c["is_weekend"],
            "dow": c["dow"],
        }])

    # Case A: predicting a date within history
    if target_ts <= last_known:
        t = target_ts - pd.Timedelta(days=1)
        sub = hist.loc[hist["date"] <= t].tail(28)
        if sub.empty:
            raise ValueError("Not enough history before that date to build features.")
        recent_vals[:] = list(sub["cash_out"].astype(float).values)
        X = make_feature_row(t)
        p50 = float(max(0.0, pipe.predict(X)[0]))
        p90 = p50 * (1 + P90_BUFFER_REL)
        return p50, p90, last_known.date(), 1

    # Case B: multi-step forecast beyond last_known
    cur = last_known
    p50 = None
    while cur < target_ts:
        X = make_feature_row(cur)     # predict for cur+1
        p50 = float(max(0.0, pipe.predict(X)[0]))
        recent_vals.append(p50)
        if len(recent_vals) > 28:
            recent_vals.pop(0)
        cur = cur + pd.Timedelta(days=1)

    steps_ahead = int((target_ts - last_known).days)
    p90 = p50 * (1 + P90_BUFFER_REL)
    return p50, p90, last_known.date(), steps_ahead

# --- Interactive prompts ---
branch = input("Enter branch_id (e.g., B001): ").strip()
target = input("Predict for date (YYYY-MM-DD): ").strip()

p50, p90, last_obs, k = predict_for_branch_date(branch, target)
print(f"\nLast observed date for {branch}: {last_obs}")
print(f"Predicting {k}-day(s) ahead for {branch} on {pd.to_datetime(target).date()}:")
print(f"Estimated cash needed — P50: {p50:,.0f}   P90: {p90:,.0f}")

P90 buffer (90th pct relative error) = 21.6%


In [ ]:
# =============================================================================
# CHEAT SHEET — Which model should I use for tabular forecasting?
#
# Always start with SIMPLE BASELINES you must beat:
#   - lag1  : "tomorrow = yesterday"
#   - MA7   : "tomorrow = average of last 7 days"
#
# Linear (simple & explainable)
#   - ElasticNet
#   Use when: relationships are mostly linear and you want clear coefficients.
#   Pros: fast, easy to explain. Cons: misses non-linear patterns.
#
# Trees (handle non-linearities automatically)
#   - RandomForestRegressor
#     Use when: you want a robust, low-tuning upgrade over linear models.
#   - HistGradientBoostingRegressor (built-in sklearn)
#     Use when: you want strong accuracy on tabular data with minimal deps.
#   - XGBoost (xgboost library)
#     Use when: you have thousands+ rows and want state-of-the-art accuracy.
#     Tip: use early stopping (needs a small validation slice).
#
# Classical single-series models (per branch)
#   - ARIMA / ETS / Prophet
#     Use when: modeling ONE series with strong seasonality and few features.
#     (Less convenient when you have many branches + extra regressors.)
#
# Neural nets
#   - Usually overkill for small/medium tabular datasets. Consider only if
#     you have lots of data and complex signals (images/text, long horizons).
#
# How to choose (super short):
#   1) Build features → do a time-based split.
#   2) Train ElasticNet + RandomForest + HistGB (and XGBoost if installed).
#   3) Compare WMAPE/MAE to lag1 & MA7. 
#   4) Pick the SIMPLEST model that clearly wins. If gains are tiny, prefer simpler.
# =============================================================================

# quick bake-off you can run right away (uses your existing X_train/X_test etc.)
# ---- Bake-off: ElasticNet, RandomForest, HistGB, and XGBoost (robust early stopping) ----
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

num_cols = ["lag1","lag7","ma7","ma28","is_holiday","is_payday","is_month_end","is_weekend"]
cat_cols = ["dow"]

pre = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
])

def wmape(y, yhat, eps=1e-6):
    y = np.asarray(y); yhat = np.asarray(yhat)
    return float(np.sum(np.abs(y - yhat)) / max(np.sum(np.abs(y)), eps))

results = {}

# -------- Linear & tree models via Pipeline (no special eval_set needed) --------
for name, model in {
    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0),
    "RandomForest": RandomForestRegressor(n_estimators=300, random_state=0, n_jobs=-1),
    "HistGB": HistGradientBoostingRegressor(random_state=0),
}.items():
    pipe = Pipeline([("pre", pre), ("model", model)])
    pipe.fit(X_train, y_train)
    yhat = pipe.predict(X_test)
    results[name] = {"WMAPE": round(wmape(y_test, yhat), 4),
                     "MAE":   round(float(mean_absolute_error(y_test, yhat)), 2)}

# -------- XGBoost with robust early-stopping (no Pipeline kwargs) --------
# -------- XGBoost with version-safe early stopping --------
try:
    import xgboost as xgb, inspect

    # make a small validation slice at the tail of training
    VAL_DAYS = 30
    val_start = test_start - pd.Timedelta(days=VAL_DAYS)
    train_core = (dfm["date"] <= val_start)
    val_core   = (dfm["date"] >  val_start) & (dfm["date"] <= test_start)

    X_tr, y_tr = dfm.loc[train_core, num_cols+cat_cols], dfm.loc[train_core, "y_next"]
    X_val, y_val = dfm.loc[val_core,  num_cols+cat_cols], dfm.loc[val_core,  "y_next"]

    # fit preprocessor once and transform all splits
    pre_xgb = pre.fit(X_tr)
    Xt_tr   = pre_xgb.transform(X_tr)
    Xt_val  = pre_xgb.transform(X_val)
    Xt_test = pre_xgb.transform(X_test)

    xgb_model = xgb.XGBRegressor(
        n_estimators=2000, learning_rate=0.03, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
        min_child_weight=1, tree_method="hist", random_state=0, n_jobs=-1
    )

    # choose a compatible early-stopping API
    params = inspect.signature(xgb_model.fit).parameters
    if "callbacks" in params:  # newer API
        cb = [xgb.callback.EarlyStopping(rounds=50, save_best=True)]
        xgb_model.fit(Xt_tr, y_tr.values, eval_set=[(Xt_val, y_val.values)],
                      callbacks=cb, verbose=False)
    elif "early_stopping_rounds" in params:  # older-but-common API
        xgb_model.fit(Xt_tr, y_tr.values, eval_set=[(Xt_val, y_val.values)],
                      early_stopping_rounds=50, verbose=False)
    else:  # very old API: no early stopping
        xgb_model.fit(Xt_tr, y_tr.values)

    yhat_xgb = xgb_model.predict(Xt_test)
    results["XGBoost"] = {
        "WMAPE": round(wmape(y_test, yhat_xgb), 4),
        "MAE":   round(float(mean_absolute_error(y_test, yhat_xgb)), 2),
    }
except Exception as e:
    results["XGBoost"] = {"error": f"Skipped XGBoost: {e}"}

# -------- Baselines for context --------
naive_pred = X_test["lag1"].to_numpy()
ma7_pred   = X_test["ma7"].to_numpy()
results["Baseline_lag1"] = {"WMAPE": round(wmape(y_test, naive_pred), 4),
                            "MAE":   round(float(mean_absolute_error(y_test, naive_pred)), 2)}
results["Baseline_MA7"]  = {"WMAPE": round(wmape(y_test, ma7_pred), 4),
                            "MAE":   round(float(mean_absolute_error(y_test, ma7_pred)), 2)}

results  # pick the smallest WMAPE/MAE; prefer the simpler model if scores are close

# WMAPE — Weighted Mean Absolute Percentage Error - total error as a percent of total actuals - Overall we’re off by 15%
# MAE — Mean Absolute Error - average dollar mistake Ex - On average, we’re off by $X per day/row